In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Training with PyTorch Lightning

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/training/pytorch_lightning_custom_container_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/training/pytorch_lightning_custom_container_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/pytorch_lightning_custom_container_training.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
     </a>
  </td>
</table>

## Overview

This tutorial demonstrates how to use the Vertex AI SDK for Python to train a ResNet model using custom containers and PyTorch Lightning. The model training code is from the CIFAR-10 training example on PyTorch Lightning's documentation page: 
https://pytorch-lightning.readthedocs.io/en/stable/notebooks/lightning_examples/cifar10-baseline.html

Two training approaches are taken: 1) Multiple GPU training on a single machine 2) Multiple machine training wtih a single GPU on each machine.

Learn more about [Custom Training Overview](https://cloud.google.com/vertex-ai/docs/training/overview).

### Objective

In this notebook, you learn how to take an existing example of a model trained using PyTorch Lighting, and use Vertex AI to distribute training across GPUs and multiple machines.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Training`
- `Vertex AI TensorBoard`

The steps performed include:

    * Install and import libraries to test model training locally
    * Initialize the Vertex AI SDK
    * Create a custom container for training
    * Create a Vertex AI TensorBoard
    * Modify the code for pass in arguments, log to the TensorBoard, and save the model to Cloud Storage
    * Run a Vertex AI training job on a single machine with GPUs
    * Run a Vertex AI training job on multiple machines with single GPUs attached

### Dataset

Here's the description from the website: The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.
https://www.cs.toronto.edu/~kriz/cifar.html

The dataset will be loaded using the Lightning Bolts datamodules.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage. 

### Install additional packages

Install additional package dependencies not installed in your notebook environment

In [ ]:
USER_FLAG = ""

! pip3 install {USER_FLAG} --upgrade "torch>=1.6, <1.9"
! pip3 install {USER_FLAG} --upgrade "lightning-bolts"
! pip3 install {USER_FLAG} --upgrade git+https://github.com/PyTorchLightning/pytorch-lightning
! pip3 install {USER_FLAG} --upgrade "torchmetrics>=0.3"
! pip3 install {USER_FLAG} --upgrade "torchvision"
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform
! pip3 install {USER_FLAG} --upgrade ipywidgets

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Set the region

**Optional**: Update the 'REGION' variable to specify the region that you want to use. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append the uuid onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

To authenticate your Google Cloud account, follow the instructions for your Jupyter environment:

**1. Vertex AI Workbench**
<br>You are already authenticated.

**2. Local JupyterLab instance**
<br>Uncomment and run the following code:

In [ ]:
# ! gcloud auth login

**3. Colab**
<br>Uncomment and run the following code:

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # noqa

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Import libraries and define constants

In [ ]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy

seed_everything(7)

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 2)

print(PATH_DATASETS)
print(AVAIL_GPUS)
print(BATCH_SIZE)
print(NUM_WORKERS)

### Define training functions for local testing

In [ ]:
train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

cifar10_dm = CIFAR10DataModule(
    data_dir=PATH_DATASETS,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    val_transforms=test_transforms,
)


def create_model():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(
        3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    )
    model.maxpool = nn.Identity()
    return model


class LitResnet(LightningModule):
    def __init__(self, lr=0.05):
        super().__init__()

        self.save_hyperparameters()
        self.model = create_model()

    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.lr,
            momentum=0.9,
            weight_decay=5e-4,
        )
        steps_per_epoch = 45000 // BATCH_SIZE
        scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}

#### Train the model locally

In [ ]:
model = LitResnet(lr=0.05)
model.datamodule = cifar10_dm

trainer = Trainer(
    max_epochs=5,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger("lightning_logs/", name="resnet"),
    callbacks=[LearningRateMonitor(logging_interval="step")],
    strategy="dp",
)

trainer.fit(model, cifar10_dm)
trainer.test(model, datamodule=cifar10_dm)

## Vertex AI Training using the Vertex AI SDK and a custom container

### Build the custom container

#### Run these steps once to setup artifact registry and authorize docker to use it

In [ ]:
! gcloud config set project $PROJECT_ID
! gcloud services enable artifactregistry.googleapis.com
! sudo usermod -a -G docker ${USER}
! gcloud auth configure-docker us-central1-docker.pkg.dev --quiet

In [ ]:
REPOSITORY = "gpu-training-repository"

In [ ]:
! gcloud artifacts repositories create $REPOSITORY --repository-format=docker \
--location=$REGION --description="Vertex GPU training repository"

#### Make a trainer directory

In [ ]:
import os

os.mkdir("trainer")

#### Build the container
This code extends the original example and adds argument parsing, TensorBoard logging, ability to choose the training strategy, and model saving to Cloud Storage 

In [ ]:
%%writefile trainer/task.py
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy

# Arg parsing and shutil for folder creation
import argparse
import shutil

seed_everything(7)

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 2)

print (PATH_DATASETS)
print (AVAIL_GPUS)
print (BATCH_SIZE)
print (NUM_WORKERS)

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

cifar10_dm = CIFAR10DataModule(
    data_dir=PATH_DATASETS,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    val_transforms=test_transforms,
)

# Added code to read args
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--epochs', dest='epochs',
                        default=10, type=int,
                        help='Number of epochs.')
    parser.add_argument('--distribute', dest='distribute', type=str, default='dp',
                        help='Distributed training strategy.')
    parser.add_argument('--num-nodes', dest='num_nodes',
                        default=1, type=int,
                        help='Number of nodes')
    parser.add_argument(
          '--model-dir', dest='model_dir', default=os.getenv('AIP_MODEL_DIR'), type=str,
          help='a Cloud Storage URI of a directory intended for saving model artifacts')
    parser.add_argument(
          '--tensorboard-log-dir', dest='tensorboard_log_dir', default=os.getenv('AIP_TENSORBOARD_LOG_DIR'), type=str,
          help='a Cloud Storage URI of a directory intended for saving TensorBoard')
    parser.add_argument(
          '--checkpoint-dir', dest='checkpoint_dir', default=os.getenv('AIP_CHECKPOINT_DIR'), type=str,
          help='a Cloud Storage URI of a directory intended for saving checkpoints')
    args = parser.parse_args()
    return args

# Cunction to make model directory if it doesn't exist
def makedirs(model_dir):
    if os.path.exists(model_dir) and os.path.isdir(model_dir):
        shutil.rmtree(model_dir)
    os.makedirs(model_dir)
    return

def create_model():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

class LitResnet(LightningModule):
    def __init__(self, lr=0.05):
        super().__init__()

        self.save_hyperparameters()
        self.model = create_model()

    # TensorBoard logging at epoch end
    def training_epoch_end(self,outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()

        tensorboard_logs = {'loss': avg_loss}

        epoch_dictionary={'loss': avg_loss,'log': tensorboard_logs}

    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)

        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)


    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.lr,
            momentum=0.9,
            weight_decay=5e-4,
        )
        steps_per_epoch = 45000 // BATCH_SIZE
        scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}

def main():   

    # Parse args
    args = parse_args()
    print (f"Args={args}")
    print (f"model directory={args.epochs}")
    print (f"model directory={args.model_dir}")
    print (f"distribute strategy={args.distribute}")

    # model, ensorboard, and checkpoint directories set
    local_model_dir = './tmp/model'
    local_tensorboard_log_dir = './tmp/logs'
    local_checkpoint_dir = './tmp/checkpoints'

    model_dir = args.model_dir or local_model_dir
    tensorboard_log_dir = args.tensorboard_log_dir or local_tensorboard_log_dir
    checkpoint_dir = args.checkpoint_dir or local_checkpoint_dir

    print ("Model directory" + model_dir)
    print ("TensorBoard directory" + tensorboard_log_dir)
    print ("Checkpoint directory" + checkpoint_dir)

    gs_prefix = 'gs://'
    gcsfuse_prefix = '/gcs/'
    if model_dir and model_dir.startswith(gs_prefix):
        model_dir = model_dir.replace(gs_prefix, gcsfuse_prefix)
        if not os.path.isdir(model_dir):
            os.makedirs(model_dir)
    if tensorboard_log_dir and tensorboard_log_dir.startswith(gs_prefix):
        tensorboard_log_dir = tensorboard_log_dir.replace(gs_prefix, gcsfuse_prefix)
        if not os.path.isdir(tensorboard_log_dir):
            os.makedirs(tensorboard_log_dir)
    if checkpoint_dir and checkpoint_dir.startswith(gs_prefix):
        checkpoint_dir = checkpoint_dir.replace(gs_prefix, gcsfuse_prefix)
        if not os.path.isdir(checkpoint_dir):
            os.makedirs(checkpoint_dir)

    model = LitResnet(lr=0.05)
    model.datamodule = cifar10_dm

    trainer = Trainer(
        progress_bar_refresh_rate=10,
        gpus=AVAIL_GPUS, 
        logger=TensorBoardLogger(tensorboard_log_dir, "resnet"), 
        callbacks=[LearningRateMonitor(logging_interval="step")],
        # Changes to use args, change default checkpoint dir, and set number of nodes
        max_epochs=args.epochs,
        strategy=args.distribute,
        default_root_dir=checkpoint_dir,
        num_nodes=args.num_nodes,
    )

    trainer.fit(model, cifar10_dm)
    trainer.test(model, datamodule=cifar10_dm)

    #Save model step
    model_name = "pylightning_resnet_state_dict.pth"

    model_save_path = os.path.join(model_dir, model_name)
    if trainer.global_rank == 0:
        makedirs(model_dir)
        print("Saving model to {}".format(model_save_path))
        torch.save(model.state_dict(), model_save_path)


if __name__ == '__main__':
    main()

#### Configure the container name and path to artifact registry

In [ ]:
content_name = "pytorch-lightning-gpu-training"
hostname = f"{REGION}-docker.pkg.dev"
image_name_train = content_name
tag = "latest"

custom_container_image_uri_train = (
    f"{hostname}/{PROJECT_ID}/{REPOSITORY}/{image_name_train}:{tag}"  # noqa
)

#### Create the requirements.txt and Dockerfile

In [ ]:
%%writefile trainer/requirements.txt
torch>=1.6, <1.9
lightning-bolts
pytorch-lightning=1.3
torchmetrics>=0.3
torchvision

In [ ]:
%%writefile trainer/Dockerfile
FROM pytorch/pytorch:1.8.1-cuda11.1-cudnn8-runtime

COPY . /trainer

WORKDIR /trainer

RUN pip install -r requirements.txt

ENTRYPOINT ["python", "task.py"]

#### Create an empty __init__.py file required to be in the container

In [ ]:
import os

with open(os.path.join("trainer", "__init__.py"), "w") as fp:
    pass

#### Build the container, train the model within the container image locally, and push to Artifact Registry

In [ ]:
! cd trainer && docker build -t $custom_container_image_uri_train -f Dockerfile .

In [ ]:
! docker run --rm $custom_container_image_uri_train

In [ ]:
! docker push $custom_container_image_uri_train

In [ ]:
! gcloud artifacts repositories describe $REPOSITORY --location=$REGION

### Initialize Vertex SDK

In [ ]:
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_URI,
    location=REGION,
)

### Create a Vertex AI TensorBoard Instance

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=content_name,
)

#### Option: Use a previously created Vertex AI TensorBoard instance

```
tensorboard_name = "Your TensorBoard Resource Name or TensorBoard ID"
tensorboard = aiplatform.Tensorboard(tensorboard_name=tensorboard_name)
```

#### Set the parameters for the training. The model/TensorBoard/checkpoint directory uses the Vertex defaults. Uncomment to set your own

In [ ]:
gcs_output_uri_prefix = f"{BUCKET_URI}/{content_name}-{UUID}"

In [ ]:
EPOCHS = 30
TRAIN_STRATEGY = "dp"  # Distributed Parallel for single machine multiple GPU
MODEL_DIR = f"{BUCKET_URI}/{content_name}/model"
TB_DIR = f"{BUCKET_URI}/{content_name}/logs"
CHKPT_DIR = f"{BUCKET_URI}/{content_name}/checkpoints"
NUM_NODES = 1

machine_type = "n1-standard-4"
accelerator_count = 2
accelerator_type = "NVIDIA_TESLA_V100"

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--distribute=" + TRAIN_STRATEGY,
    "--num-nodes=" + str(NUM_NODES),
    "--model-dir=" + MODEL_DIR,
    "--checkpoint-dir=" + CHKPT_DIR,
]

In [ ]:
custom_container_training_job = aiplatform.CustomContainerTrainingJob(
    display_name=content_name + "-MultGPU-dp-" + UUID,
    container_uri=custom_container_image_uri_train,
)

In [ ]:
custom_container_training_job.run(
    args=CMDARGS,
    replica_count=NUM_NODES,
    base_output_dir=gcs_output_uri_prefix,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    service_account=SERVICE_ACCOUNT,
    tensorboard=tensorboard.resource_name,
    sync=False,
)

In [ ]:
print(f"Custom Training Job Name: {custom_container_training_job.resource_name}")
print(f"GCS Output URI Prefix: {gcs_output_uri_prefix}")

#### Delete the training job

In [ ]:
custom_container_training_job.delete()

## Run training on multiple machines w/ 1 GPU on each

In [ ]:
EPOCHS = 30
TRAIN_STRATEGY = "ddp"  # Distributed Parallel for single machine multiple GPU
MODEL_DIR = f"{BUCKET_URI}/{content_name}-ddp/model"
TB_DIR = f"{BUCKET_URI}/{content_name}-ddp/logs"
CHKPT_DIR = f"{BUCKET_URI}/{content_name}-ddp/checkpoints"
NUM_NODES = 2

machine_type = "n1-standard-4"
accelerator_count = 1
accelerator_type = "NVIDIA_TESLA_V100"

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--distribute=" + TRAIN_STRATEGY,
    "--num-nodes=" + str(NUM_NODES),
    "--model-dir=" + MODEL_DIR,
    "--checkpoint-dir=" + CHKPT_DIR,
]

In [ ]:
custom_container_training_job_dist = aiplatform.CustomContainerTrainingJob(
    display_name=content_name + "-MultiCPU-1GPU-ddp-" + UUID,
    container_uri=custom_container_image_uri_train,
)

In [ ]:
custom_container_training_job_dist.run(
    args=CMDARGS,
    replica_count=NUM_NODES,
    base_output_dir=gcs_output_uri_prefix,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    service_account=SERVICE_ACCOUNT,
    tensorboard=tensorboard.resource_name,
    sync=False,
)

In [ ]:
print(f"Custom Training Job Name: {custom_container_training_job_dist.resource_name}")
print(f"GCS Output URI Prefix: {gcs_output_uri_prefix}")

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete TensorBoard
TB_NAME = tensorboard.resource_name
! gcloud beta ai tensorboards delete $TB_NAME --quiet

# Delete the training job
custom_container_training_job_dist.delete()

CONTENT_DIR = f"{BUCKET_URI}/{content_name}*"
# Delete Cloud Storage objects that were created
! gsutil -m rm -r $CONTENT_DIR

if delete_bucket and "BUCKET_URI" in globals():
    ! gsutil -m rm -r $BUCKET_URI